<a href="https://colab.research.google.com/github/AmitNikhade/AmitNikhade/blob/main/T5_finetuning_qa_closedbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # !pip install sentencepiece
# !pip uninstall transformers==4.19.2
!pip install transformers==4.18.0
# !pip uninstall sentencepiece -y

import transformers
# # import sentencepiece
transformers.__version__
# # sentencepiece.__version__
!pip install sentencepiece==0.1.96
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 66.6 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
     |████████████████████████████████| 6.6 MB 55.2 MB/s 
     |████████████████████████████████| 880 kB 50.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4359af7047fcda7b38c84029bf560c7cd7e3b7209ad215b6c191b76729ee102d
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 5.1 MB/s 
Looking in i

In [146]:
import torch
# import nlp
from transformers import AutoTokenizer
import transformers
# from transformers import Au
tokenizer = AutoTokenizer.from_pretrained('google/t5-xxl-ssm-tqa')

In [147]:
# process the examples in input and target text format and the eos token at the end 
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s </s>' % (example['question'], example['context'])
    example['target_text'] = '%s </s>' % example['answers'][1]
    return example

# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=512)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=16)

    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [148]:
import pandas as pd
df = pd.read_csv("/content/file1.csv")
# df
df = df.drop(columns=['answers'])
df = df.drop(columns=['Unnamed: 0'])
# # # df
df['answers'] = [[str(i),j] for i,j in zip(df['answer_start'],df['answer_text'])]
# df.columns

In [149]:
import datasets
from datasets import Dataset
# df = Dataset.from_pandas(df)

In [150]:
df = Dataset.from_pandas(df)
train_dataset1 = df.map(add_eos_to_examples)

train_dataset1 = train_dataset1.map(convert_to_features, batched=True)

  0%|          | 0/9660 [00:00<?, ?ex/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [151]:
columns = ['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask']
train_dataset1.set_format(type='torch', columns=columns)

In [152]:
from torch.utils.data import Dataset, DataLoader
train_params = {
        "batch_size": 8,
        "shuffle": True,
        "num_workers": 0,
    }
training_loader = DataLoader(train_dataset1, **train_params)

In [153]:
import tqdm
def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    for _, data in tqdm.tqdm_notebook(enumerate(loader, 0)):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["input_ids"].to(device, dtype=torch.long)
        mask = data["attention_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]

        if _ % 10 == 0:
          print(str(epoch), str(_), str(loss))
        #     training_logger.add_row(str(epoch), str(_), str(loss))
        #     console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration
# model = T5ForConditionalGeneration.from_pretrained('t5-base')
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-xl-ssm-nq")
optimizer = torch.optim.Adam(
        params=model.parameters(), lr=0.001
    )

from torch import cuda

device = 'cuda' 
model = model.to(device)
  
model.train()
for epoch in range(1):
  train(epoch, tokenizer, model, device, training_loader, optimizer)

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6G [00:00<?, ?B/s]

In [4]:
# model.save_pretrained('/content/drive/MyDrive/')
# tokenizer.save_pretrained('/content/drive/MyDrive/')
model = model.from_pretrained('/content/drive/MyDrive/')
tokenizer = tokenizer.from_pretrained('/content/drive/MyDrive/')
# print(model)

In [46]:
for i in training_loader:
  # print(i['input_ids'].to(device, dtype = torch.long))
  print([tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in i['input_ids'].to(device, dtype = torch.long)])
  print([tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in i['target_ids'].to(device, dtype = torch.long)])
  break

['question: What is the name of the person who is a member of the governing body of the governing body of the governing body? context: Mr', 'question: What happened to the cupboard when the drawers were open? context: The drawers were open, the cupboard locked', 'question: What glowed amid the tangle of white hair? context: A cigarette glowed amid the tangle of white hair, and the air of the room was fetid with stale tobacco-smoke', 'question: What do new indications point to? context: Up to this point the incident bore the appearance of an ordinary accident, but fresh indications seem to point to serious crime', 'question: What was the name of the woman who tapped at the door? context: " "What happened there, madam?" "I tapped at the door as agreed', 'question: What kind of jobs were advertised in the ads? context: I saw these advertisements about harpooners and high wages, so I went to the shipping agents, and they sent me here', "question: What did she say about her own photograph? 

In [136]:
g = tokenizer.encode_plus('question: How long have I been married?',pad_to_max_length=True, max_length=50)
g

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


{'input_ids': [822, 10, 571, 307, 43, 27, 118, 4464, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [137]:
ids = torch.Tensor(g['attention_mask']).to(device, dtype = torch.long).reshape(1,len(g['attention_mask']))
ids

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]], device='cuda:0')

In [138]:
ids = torch.Tensor(g['input_ids']).to(device, dtype = torch.long).reshape(1,len(g['input_ids']))
mask = torch.Tensor(g['attention_mask']).to(device, dtype = torch.long).reshape(1,len(g['attention_mask']))
# print(ids)
# print(mask)
generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
print(generated_ids)
for i in generated_ids:
  print(tokenizer.decode(i, skip_special_tokens=True, clean_up_tokenization_spaces=True))
  break
# print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids])

tensor([[   0,   27,  118, 4464,    1]], device='cuda:0')
I been married


In [122]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  device = 'cuda'
  model.to(device)
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in tqdm.tqdm_notebook(enumerate(loader, 0)):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['input_ids'].to(device, dtype = torch.long)
          mask = data['attention_mask'].to(device, dtype = torch.long)
          print(ids)
          # break
          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          print(generated_ids)
          question = [tokenizer.decode(p, skip_special_tokens=True, clean_up_tokenization_spaces=True) for p in ids]
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          # print(tokenizer.decode(data['input_ids'], skip_special_tokens=True, clean_up_tokenization_spaces=True))
          # if _%10==0:
          #     console.print(f'Completed {_}')
          # print("preds",preds)
          # print("actual:",actuals)
          print(question)
          predictions.extend(preds)
          actuals.extend(target)
          break
  # print(predictions, actuals)
  return predictions, actuals

In [123]:
validate(1, tokenizer, model, device, training_loader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


0it [00:00, ?it/s]

tensor([[ 822,   10,  571,  ...,    0,    0,    0],
        [ 822,   10, 3963,  ...,    0,    0,    0],
        [ 822,   10,  571,  ...,    0,    0,    0],
        ...,
        [ 822,   10,  571,  ...,    0,    0,    0],
        [ 822,   10,  363,  ...,    0,    0,    0],
        [ 822,   10, 2645,  ...,    0,    0,    0]], device='cuda:0')
tensor([[    0,     3,     9,   215,     1,     0,     0,     0,     0],
        [    0, 10235,     1,     0,     0,     0,     0,     0,     0],
        [    0,    27,   278,    31,    17, 10419,  1286,    34,     1],
        [    0,  6393,  8745,     1,     0,     0,     0,     0,     0],
        [    0, 11957,  2126,     1,     0,     0,     0,     0,     0],
        [    0,     3, 17265,     1,     0,     0,     0,     0,     0],
        [    0,    96,   683,     9,   509,   115,     7,     1,     0],
        [    0,    27,     1,     0,     0,     0,     0,     0,     0]],
       device='cuda:0')
['question: How long have I been married? contex

(['a year',
  'horses',
  "I don't insist upon it",
  'Dunlop',
  'Foreign Office',
  'heir',
  '"Jacobs',
  'I'],
 ['about a year',
  'impressed you',
  "I don't insist upon it",
  'Dunlop',
  'Persia',
  'closely related',
  'bring down my despatch-box',
  'I'])